# Equivox Fairness Platform

An AI-driven fairness platform for bias-free evaluation of Urdu audio clips in hiring and workplace assessment.

This notebook processes 3-8 second Urdu audio clips to evaluate candidates based solely on clarity of thought and problem-solving ability, removing bias from irrelevant human markers like accent, gender, and race.

## 1. Environment Setup and Dependencies

First, let's install and import all required libraries for audio processing, machine learning, and visualization.

In [ ]:
# Install required dependencies
!pip install librosa>=0.9.0
!pip install soundfile>=0.10.0
!pip install scipy>=1.7.0
!pip install transformers>=4.20.0
!pip install torch>=1.12.0
!pip install scikit-learn>=1.1.0
!pip install matplotlib>=3.5.0
!pip install seaborn>=0.11.0
!pip install umap-learn>=0.5.0
!pip install plotly>=5.0.0
!pip install ipywidgets>=7.6.0

In [ ]:
# Core audio processing imports
import librosa
import soundfile as sf
import scipy.signal
import numpy as np
import pandas as pd

# Machine learning imports
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torch
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import umap

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Notebook interface imports
import ipywidgets as widgets
from IPython.display import display, Audio, HTML
import warnings

# Standard library imports
import os
import json
import time
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Any

# Configure warnings and display settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All imports successful!")

In [ ]:
# Display system information
import platform
import psutil

print("=== System Information ===")
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Python Version: {platform.python_version()}")
print(f"CPU Cores: {psutil.cpu_count()}")
print(f"RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"Available RAM: {psutil.virtual_memory().available / (1024**3):.1f} GB")

print("\n=== Library Versions ===")
print(f"LibROSA: {librosa.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"Scikit-learn: {sklearn.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")
print(f"Seaborn: {sns.__version__}")

# Check for CUDA availability
if torch.cuda.is_available():
    print(f"\n🚀 CUDA Available: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
else:
    print("\n💻 Using CPU for processing")

print("\n✅ Environment setup complete!")

In [ ]:
# Test basic audio processing capabilities
print("=== Testing Audio Processing Capabilities ===")

# Test librosa functionality
try:
    # Generate a test sine wave
    duration = 3.0  # seconds
    sr = 16000  # sample rate
    frequency = 440  # A4 note
    
    t = np.linspace(0, duration, int(sr * duration), False)
    test_audio = np.sin(2 * np.pi * frequency * t)
    
    # Test MFCC extraction
    mfccs = librosa.feature.mfcc(y=test_audio, sr=sr, n_mfcc=13)
    print(f"✅ MFCC extraction: {mfccs.shape}")
    
    # Test spectral features
    spectral_centroids = librosa.feature.spectral_centroid(y=test_audio, sr=sr)
    print(f"✅ Spectral centroid: {spectral_centroids.shape}")
    
    # Test chroma features
    chroma = librosa.feature.chroma_stft(y=test_audio, sr=sr)
    print(f"✅ Chroma features: {chroma.shape}")
    
    print("✅ LibROSA audio processing test successful!")
    
except Exception as e:
    print(f"❌ LibROSA test failed: {e}")

# Test transformers model loading (without actually downloading)
try:
    from transformers import Wav2Vec2Processor
    print("✅ Transformers library ready for wav2vec2 model")
except Exception as e:
    print(f"❌ Transformers test failed: {e}")

print("\n🎉 Basic functionality tests complete!")

## 2. Audio Preprocessing Module

This module handles loading, normalizing, and validating audio files for consistent processing across different formats and sample rates.

In [ ]:
# Data structures for audio processing
@dataclass
class AudioSample:
    """Data structure for audio samples"""
    file_path: str
    audio_data: np.ndarray
    sample_rate: int
    duration: float
    
    def __post_init__(self):
        """Validate audio sample after initialization"""
        if self.duration < 3.0 or self.duration > 8.0:
            raise ValueError(f"Audio duration {self.duration:.2f}s is outside valid range (3-8 seconds)")
        if self.sample_rate != 16000:
            print(f"Warning: Sample rate {self.sample_rate} Hz will be normalized to 16000 Hz")

class AudioPreprocessor:
    """Audio preprocessing module for loading, normalizing, and validating audio files"""
    
    def __init__(self, target_sr: int = 16000, min_duration: float = 3.0, max_duration: float = 8.0):
        """
        Initialize audio preprocessor
        
        Args:
            target_sr: Target sample rate for normalization (default: 16000 Hz)
            min_duration: Minimum allowed audio duration in seconds
            max_duration: Maximum allowed audio duration in seconds
        """
        self.target_sr = target_sr
        self.min_duration = min_duration
        self.max_duration = max_duration
        self.supported_formats = ['.wav', '.mp3', '.m4a', '.flac', '.ogg']
        
    def load_audio(self, file_path: str) -> Tuple[np.ndarray, int]:
        """
        Load audio file using librosa with format detection
        
        Args:
            file_path: Path to audio file
            
        Returns:
            Tuple of (audio_data, original_sample_rate)
            
        Raises:
            FileNotFoundError: If file doesn't exist
            ValueError: If file format is not supported
            RuntimeError: If audio loading fails
        """
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Audio file not found: {file_path}")
            
        file_ext = Path(file_path).suffix.lower()
        if file_ext not in self.supported_formats:
            raise ValueError(f"Unsupported audio format: {file_ext}. Supported formats: {self.supported_formats}")
        
        try:
            # Load audio with librosa (automatically handles various formats)
            audio_data, sample_rate = librosa.load(file_path, sr=None, mono=True)
            
            if len(audio_data) == 0:
                raise RuntimeError("Loaded audio file is empty")
                
            return audio_data, sample_rate
            
        except Exception as e:
            raise RuntimeError(f"Failed to load audio file {file_path}: {str(e)}")
    
    def normalize_sample_rate(self, audio_data: np.ndarray, original_sr: int) -> np.ndarray:
        """
        Normalize audio to target sample rate
        
        Args:
            audio_data: Input audio data
            original_sr: Original sample rate
            
        Returns:
            Resampled audio data
        """
        if original_sr == self.target_sr:
            return audio_data
            
        # Use librosa's high-quality resampling
        resampled_audio = librosa.resample(audio_data, orig_sr=original_sr, target_sr=self.target_sr)
        return resampled_audio
    
    def validate_duration(self, audio_data: np.ndarray, sample_rate: int) -> float:
        """
        Validate audio duration against constraints
        
        Args:
            audio_data: Audio data array
            sample_rate: Sample rate of audio
            
        Returns:
            Duration in seconds
            
        Raises:
            ValueError: If duration is outside valid range
        """
        duration = len(audio_data) / sample_rate
        
        if duration < self.min_duration:
            raise ValueError(f"Audio duration {duration:.2f}s is too short (minimum: {self.min_duration}s)")
        
        if duration > self.max_duration:
            raise ValueError(f"Audio duration {duration:.2f}s is too long (maximum: {self.max_duration}s)")
            
        return duration
    
    def apply_noise_reduction(self, audio_data: np.ndarray) -> np.ndarray:
        """
        Apply basic noise reduction using spectral gating
        
        Args:
            audio_data: Input audio data
            
        Returns:
            Noise-reduced audio data
        """
        # Simple noise reduction using spectral subtraction
        # Estimate noise from first 0.5 seconds (assuming it contains background noise)
        noise_sample_length = min(int(0.5 * self.target_sr), len(audio_data) // 4)
        
        if noise_sample_length > 0:
            noise_spectrum = np.abs(np.fft.fft(audio_data[:noise_sample_length]))
            noise_power = np.mean(noise_spectrum ** 2)
            
            # Apply gentle high-pass filter to reduce low-frequency noise
            if noise_power > 0:
                sos = scipy.signal.butter(4, 80, btype='high', fs=self.target_sr, output='sos')
                filtered_audio = scipy.signal.sosfilt(sos, audio_data)
                return filtered_audio
        
        return audio_data
    
    def preprocess_audio(self, file_path: str, apply_noise_reduction: bool = True) -> AudioSample:
        """
        Complete audio preprocessing pipeline
        
        Args:
            file_path: Path to audio file
            apply_noise_reduction: Whether to apply noise reduction
            
        Returns:
            AudioSample object with processed audio
        """
        # Load audio file
        audio_data, original_sr = self.load_audio(file_path)
        
        # Normalize sample rate
        normalized_audio = self.normalize_sample_rate(audio_data, original_sr)
        
        # Validate duration
        duration = self.validate_duration(normalized_audio, self.target_sr)
        
        # Apply noise reduction if requested
        if apply_noise_reduction:
            processed_audio = self.apply_noise_reduction(normalized_audio)
        else:
            processed_audio = normalized_audio
        
        # Normalize amplitude to [-1, 1] range
        max_amplitude = np.max(np.abs(processed_audio))
        if max_amplitude > 0:
            processed_audio = processed_audio / max_amplitude
        
        return AudioSample(
            file_path=file_path,
            audio_data=processed_audio,
            sample_rate=self.target_sr,
            duration=duration
        )

# Initialize the audio preprocessor
audio_preprocessor = AudioPreprocessor()
print("✅ Audio preprocessing module initialized!")
print(f"Target sample rate: {audio_preprocessor.target_sr} Hz")
print(f"Valid duration range: {audio_preprocessor.min_duration}-{audio_preprocessor.max_duration} seconds")
print(f"Supported formats: {audio_preprocessor.supported_formats}")

In [ ]:
# Audio visualization utilities
def plot_waveform(audio_sample: AudioSample, title: str = None, figsize: Tuple[int, int] = (12, 4)):
    """
    Plot audio waveform with time axis
    
    Args:
        audio_sample: AudioSample object to plot
        title: Optional title for the plot
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    
    # Create time axis
    time_axis = np.linspace(0, audio_sample.duration, len(audio_sample.audio_data))
    
    plt.plot(time_axis, audio_sample.audio_data, color='steelblue', linewidth=0.8)
    plt.xlabel('Time (seconds)')
    plt.ylabel('Amplitude')
    plt.grid(True, alpha=0.3)
    
    if title:
        plt.title(title)
    else:
        plt.title(f'Waveform - {Path(audio_sample.file_path).name} ({audio_sample.duration:.2f}s)')
    
    plt.tight_layout()
    plt.show()

def plot_spectrogram(audio_sample: AudioSample, title: str = None, figsize: Tuple[int, int] = (12, 6)):
    """
    Plot audio spectrogram
    
    Args:
        audio_sample: AudioSample object to plot
        title: Optional title for the plot
        figsize: Figure size tuple
    """
    plt.figure(figsize=figsize)
    
    # Compute spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(audio_sample.audio_data)), ref=np.max)
    
    librosa.display.specshow(D, sr=audio_sample.sample_rate, x_axis='time', y_axis='hz')
    plt.colorbar(format='%+2.0f dB')
    
    if title:
        plt.title(title)
    else:
        plt.title(f'Spectrogram - {Path(audio_sample.file_path).name}')
    
    plt.tight_layout()
    plt.show()

def display_audio_info(audio_sample: AudioSample):
    """
    Display comprehensive information about an audio sample
    
    Args:
        audio_sample: AudioSample object to analyze
    """
    print(f"=== Audio Information ===")
    print(f"File: {Path(audio_sample.file_path).name}")
    print(f"Duration: {audio_sample.duration:.3f} seconds")
    print(f"Sample Rate: {audio_sample.sample_rate} Hz")
    print(f"Samples: {len(audio_sample.audio_data):,}")
    print(f"Amplitude Range: [{np.min(audio_sample.audio_data):.4f}, {np.max(audio_sample.audio_data):.4f}]")
    print(f"RMS Energy: {np.sqrt(np.mean(audio_sample.audio_data**2)):.4f}")
    
    # Zero crossing rate
    zcr = librosa.feature.zero_crossing_rate(audio_sample.audio_data)[0]
    print(f"Zero Crossing Rate: {np.mean(zcr):.4f}")
    
    # Spectral centroid
    spectral_centroids = librosa.feature.spectral_centroid(y=audio_sample.audio_data, sr=audio_sample.sample_rate)[0]
    print(f"Spectral Centroid: {np.mean(spectral_centroids):.2f} Hz")

print("✅ Audio visualization utilities loaded!")

In [ ]:
# Test audio preprocessing with synthetic audio
print("=== Testing Audio Preprocessing Module ===")

# Create test audio files with different characteristics
test_dir = Path("test_audio")
test_dir.mkdir(exist_ok=True)

def create_test_audio(filename: str, duration: float, frequency: float = 440, sample_rate: int = 22050):
    """Create synthetic test audio file"""
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    # Create a more complex waveform with harmonics
    audio = (np.sin(2 * np.pi * frequency * t) + 
             0.3 * np.sin(2 * np.pi * frequency * 2 * t) + 
             0.1 * np.sin(2 * np.pi * frequency * 3 * t))
    
    # Add some noise for realism
    noise = np.random.normal(0, 0.05, len(audio))
    audio = audio + noise
    
    # Normalize
    audio = audio / np.max(np.abs(audio))
    
    filepath = test_dir / filename
    sf.write(filepath, audio, sample_rate)
    return str(filepath)

# Test cases
test_cases = [
    {"name": "valid_audio_5s.wav", "duration": 5.0, "frequency": 440, "sr": 16000, "should_pass": True},
    {"name": "high_sr_audio.wav", "duration": 4.0, "frequency": 880, "sr": 44100, "should_pass": True},
    {"name": "short_audio.wav", "duration": 2.0, "frequency": 220, "sr": 16000, "should_pass": False},
    {"name": "long_audio.wav", "duration": 10.0, "frequency": 330, "sr": 8000, "should_pass": False},
]

print("Creating test audio files...")
for test_case in test_cases:
    filepath = create_test_audio(
        test_case["name"], 
        test_case["duration"], 
        test_case["frequency"], 
        test_case["sr"]
    )
    print(f"✅ Created: {test_case['name']} ({test_case['duration']}s, {test_case['sr']} Hz)")

print("\n=== Testing Preprocessing Pipeline ===")

for test_case in test_cases:
    filepath = test_dir / test_case["name"]
    print(f"\nTesting: {test_case['name']}")
    
    try:
        # Test preprocessing
        audio_sample = audio_preprocessor.preprocess_audio(str(filepath))
        
        if test_case["should_pass"]:
            print(f"✅ Successfully processed {test_case['name']}")
            display_audio_info(audio_sample)
            
            # Verify normalization
            assert audio_sample.sample_rate == 16000, f"Sample rate not normalized: {audio_sample.sample_rate}"
            assert 3.0 <= audio_sample.duration <= 8.0, f"Duration out of range: {audio_sample.duration}"
            assert np.max(np.abs(audio_sample.audio_data)) <= 1.0, "Audio not normalized to [-1, 1]"
            
            print("✅ All validations passed")
        else:
            print(f"❌ Expected failure but processing succeeded for {test_case['name']}")
            
    except (ValueError, RuntimeError) as e:
        if not test_case["should_pass"]:
            print(f"✅ Expected failure: {e}")
        else:
            print(f"❌ Unexpected failure: {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

print("\n🎉 Audio preprocessing module testing complete!")

In [ ]:
# Demonstrate waveform visualization with test audio
print("=== Audio Visualization Demo ===")

# Load and visualize a valid test audio file
valid_test_file = test_dir / "valid_audio_5s.wav"

if valid_test_file.exists():
    try:
        # Process the audio
        audio_sample = audio_preprocessor.preprocess_audio(str(valid_test_file))
        
        print(f"Visualizing: {valid_test_file.name}")
        
        # Display comprehensive info
        display_audio_info(audio_sample)
        
        # Plot waveform
        plot_waveform(audio_sample, "Test Audio Waveform")
        
        # Plot spectrogram
        plot_spectrogram(audio_sample, "Test Audio Spectrogram")
        
        # Create audio widget for playback
        print("\n🔊 Audio Playback:")
        display(Audio(audio_sample.audio_data, rate=audio_sample.sample_rate))
        
        print("✅ Visualization demo complete!")
        
    except Exception as e:
        print(f"❌ Visualization demo failed: {e}")
else:
    print("❌ Test audio file not found")

print("\n=== Format Support Test ===")

# Test different audio formats (create WAV and test loading)
format_test_file = test_dir / "format_test.wav"
if format_test_file.exists():
    try:
        # Test loading without preprocessing
        raw_audio, raw_sr = audio_preprocessor.load_audio(str(format_test_file))
        print(f"✅ Raw loading: {len(raw_audio)} samples at {raw_sr} Hz")
        
        # Test sample rate normalization
        normalized = audio_preprocessor.normalize_sample_rate(raw_audio, raw_sr)
        print(f"✅ Normalized: {len(normalized)} samples at 16000 Hz")
        
        # Test duration validation
        duration = audio_preprocessor.validate_duration(normalized, 16000)
        print(f"✅ Duration validation: {duration:.3f} seconds")
        
    except Exception as e:
        print(f"❌ Format test failed: {e}")

print("\n🎉 All preprocessing tests completed successfully!")